# Segmenting and Clustering Neighborhoods in Toronto  
_by Francisco Peretti_

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

Your submission will be a link to your Jupyter Notebook on your Github repository.

---
# 1. Toronto Data Scraping

In [1]:
!pip install selenium
!pip install geocoder

     |████████████████████████████████| 911kB 2.7MB/s 
     |████████████████████████████████| 102kB 2.2MB/s 


In [2]:
#Import urllib which simplifies HTTP and socket management
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import re
import pandas as pd
pd.set_option('display.max_columns', None )
# pd.set_option('display.max_rows', None)

import numpy as np

import requests
import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim


In [3]:
# Web scraping

#Ignore SSl Errors
ctx = ssl.create_default_context()
ctx.check_hostname = False

# Toronto neighbouthoods Wikipedia website
# url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto'
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Retrieve HTML and parse
html = urllib.request.urlopen(url, context = ctx).read()
soup = BeautifulSoup(html, 'html.parser')

print(soup.prettify()[0:500])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"9f54


In [5]:
# Find the tables within the HTML -> Choose the second table with the required neighbourhoods data
table = soup.find_all('table', attrs={'class':'wikitable sortable'})[0]

# Select table rows
table_rows = table.find_all('tr')

l = []
head_flag = True

for tr in table_rows:

  # Use header flag to get Columns names ->
  if head_flag:
    aux = tr.find_all('th')
    header = [x.text.split('\n')[0] for x in aux]
    head_flag = False
    continue
  
  # Create list with each rows'elements
  else:
    td = tr.find_all('td')
    row = [tr.text.split('\n')[0] for tr in td]

  # Append row list and create a matrix
  l.append(row)

# Transform the table matrix into a DataFrame
toronto_df = pd.DataFrame( l )

# Set column names
toronto_df.columns = ['Postal Code','Borough','Neighborhood']

# Define Neighborhood as Borough if Neighborhood is "Not Assigned" yet Borough is
new_neigh = [ bor if (str.lower(bor) != "not assigned") & (str.lower(neigh) == "not assigned") else neigh for bor, neigh in toronto_df[['Borough','Neighborhood']].to_numpy()]
toronto_df['Neighborhood'] = new_neigh

# Drop rows with "Not Assigned" Borough
toronto_df = toronto_df[ toronto_df['Neighborhood'].str.lower() != 'not assigned' ]

# Sort values by Neighborhood
toronto_df.sort_values(['Neighborhood'], inplace = True)

# Reset index
toronto_df.reset_index(drop = True, inplace = True)

print("Shape: ", toronto_df.shape)
toronto_df.head()

Shape:  (103, 3)


,Postal Code,Borough,Neighborhood
0,M1S,Scarborough,Agincourt
1,M8W,Etobicoke,"Alderwood, Long Branch"
2,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North"
3,M2K,North York,Bayview Village
4,M5M,North York,"Bedford Park, Lawrence Manor East"


---
# 2. Data wrangle  
Find Neighborhoods' coordinates using PGeocode

In [6]:
!pip install pgeocode

In [7]:
""" Get Neighborhood' Latitude and Longitude """
import pgeocode

# Define pgeocode at "CA" for "Canada"
nomi = pgeocode.Nominatim('ca')

# Query the latitude and lonngitude of each neighborhood
lat_long = nomi.query_postal_code( toronto_df['Postal Code'].to_list() )[['postal_code','latitude','longitude']]
lat_long.rename(columns = {'postal_code':'Postal Code','latitude': 'Latitude','longitude': 'Longitude'}, inplace = True)

lat_long

,Postal Code,Latitude,Longitude
0,M1S,43.7946,-79.2644
1,M8W,43.6021,-79.5402
2,M3H,43.7535,-79.4472
3,M2K,43.7797,-79.3813
4,M5M,43.7335,-79.4177
...,...,...,...
98,M2R,43.7786,-79.4450
99,M1G,43.7712,-79.2144
100,M4C,43.6913,-79.3116
101,M2P,43.7500,-79.3978


In [8]:
# Merge original neighborhoods table with latitudes and longitudes
toronto_df = pd.merge(toronto_df, lat_long, how = 'outer', on = 'Postal Code')
toronto_df = toronto_df[~toronto_df['Latitude'].isnull()]

print('Count of missing coordinates: ', toronto_df['Latitude'].isnull().sum())
print("Shape: ", toronto_df.shape)
toronto_df.head()

Count of missing coordinates:  0
Shape:  (102, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.7946,-79.2644
1,M8W,Etobicoke,"Alderwood, Long Branch",43.6021,-79.5402
2,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.7535,-79.4472
3,M2K,North York,Bayview Village,43.7797,-79.3813
4,M5M,North York,"Bedford Park, Lawrence Manor East",43.7335,-79.4177


---
# 3. Data exploration

In [15]:
# create map of New York using latitude and longitude values

city = 'Sunnybrook, Toronto, Canada'

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_app")
latitude = geolocator.geocode(city).latitude
longitude = geolocator.geocode(city).longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11, width='70', height='70' )

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [16]:
#@title Foursquare credentials
# Hidden cell
CLIENT_ID = 'YLROMIFNBX1AXMIYRRL04CMNBU5L2KSYL5E13X0T1LHY014G' # your Foursquare ID
CLIENT_SECRET = 'TO40LHBWD5WWOJCOVFK53RDUQOCX0RDRON1JH43JYE0JIT0O' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [49]:
# From Foursquare Lab
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [50]:
toronto_venues = getNearbyVenues(names = toronto_df['Neighborhood'],
                                   latitudes = toronto_df['Latitude'],
                                   longitudes = toronto_df['Longitude']
                                  )

In [51]:
print(toronto_venues.shape)
toronto_venues.head()

(4878, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,43.7946,-79.2644,Bestco Food Market 鴻華超級市場,43.796514,-79.270790,Supermarket
1,Agincourt,43.7946,-79.2644,The Roti Hut,43.787277,-79.258724,Caribbean Restaurant
2,Agincourt,43.7946,-79.2644,Grandeur Palace 華丽宮 (Grandeur Palace 華麗宮),43.797885,-79.270585,Chinese Restaurant
3,Agincourt,43.7946,-79.2644,Yummy Cantonese Restaurant 老西関腸粉,43.787568,-79.269585,Cantonese Restaurant
4,Agincourt,43.7946,-79.2644,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot


In [52]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues.groupby('Neighborhood').count()

There are 337 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,41,41,41,41,41,41
"Alderwood, Long Branch",23,23,23,23,23,23
"Bathurst Manor, Wilson Heights, Downsview North",33,33,33,33,33,33
Bayview Village,8,8,8,8,8,8
"Bedford Park, Lawrence Manor East",41,41,41,41,41,41
...,...,...,...,...,...,...
"Willowdale, Willowdale West",17,17,17,17,17,17
Woburn,10,10,10,10,10,10
Woodbine Heights,41,41,41,41,41,41


In [53]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
criteria = [False if x == 'Neighborhood' else True for x in toronto_onehot.columns.to_list() ]
fixed_columns = ['Neighborhood'] + toronto_onehot.columns[criteria].to_list()
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Plane,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe St

In [54]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Plane,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe St

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Bakery,Sandwich Place,Coffee Shop,Sushi Restaurant,Badminton Court,Bank,Restaurant,Cantonese Restaurant
1,"Alderwood, Long Branch",Park,Pizza Place,Discount Store,Pool,Garden Center,Gas Station,Sandwich Place,Donut Shop,Liquor Store,Italian Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Coffee Shop,Bank,Gas Station,Park,Shopping Mall,French Restaurant,Supermarket,Sandwich Place,Diner
3,Bayview Village,Park,Café,Golf Course,Bank,Trail,Japanese Restaurant,Chinese Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Restaurant,Pet Store,Fast Food Restaurant,Sandwich Place,Pizza Place,Bank,Sushi Restaurant,Pub


---
# 4. Clustering

In [63]:
try:
  neighborhoods_venues_sorted.drop(columns = ['Cluster Labels'], inplace = True)
except:
  None
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(init = "k-means++", n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df.copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged[ ~toronto_merged['Cluster Labels'].isnull() ]
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

print("Cluster labels count: \n", toronto_merged.groupby('Cluster Labels')['Cluster Labels'].count())

toronto_merged.head()

Cluster labels count: 
 Cluster Labels
0    58
1     1
2    35
3     1
4     6
Name: Cluster Labels, dtype: int64


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1S,Scarborough,Agincourt,43.7946,-79.2644,2,Chinese Restaurant,Shopping Mall,Bakery,Sandwich Place,Coffee Shop,Sushi Restaurant,Badminton Court,Bank,Restaurant,Cantonese Restaurant
1,M8W,Etobicoke,"Alderwood, Long Branch",43.6021,-79.5402,2,Park,Pizza Place,Discount Store,Pool,Garden Center,Gas Station,Sandwich Place,Donut Shop,Liquor Store,Italian Restaurant
2,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.7535,-79.4472,2,Pizza Place,Coffee Shop,Bank,Gas Station,Park,Shopping Mall,French Restaurant,Supermarket,Sandwich Place,Diner
3,M2K,North York,Bayview Village,43.7797,-79.3813,4,Park,Café,Golf Course,Bank,Trail,Japanese Restaurant,Chinese Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant
4,M5M,North York,"Bedford Park, Lawrence Manor East",43.7335,-79.4177,0,Italian Restaurant,Coffee Shop,Restaurant,Pet Store,Fast Food Restaurant,Sandwich Place,Pizza Place,Bank,Sushi Restaurant,Pub


Map of Clustered Neighborhoods

In [64]:
import random 
import matplotlib.pyplot as plt 

# FUnction to generate disctinct colors for the map
def generate_colors(n): 
  rgb_values = [] 
  hex_values = [] 
  r = int(random.random() * 256) 
  g = int(random.random() * 256) 
  b = int(random.random() * 256) 
  step = 256 / n 
  for _ in range(n): 
    r += step 
    g += step 
    b += step 
    r = int(r) % 256 
    g = int(g) % 256 
    b = int(b) % 256 
    r_hex = hex(r)[2:] 
    g_hex = hex(g)[2:] 
    b_hex = hex(b)[2:] 
    hex_values.append('#' + r_hex + g_hex + b_hex) 
    rgb_values.append((r,g,b)) 
  return hex_values 


In [73]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# generate colors
colors = generate_colors( kclusters ) 

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color= colors[cluster-1],
        fill=True,
        fill_color= colors[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 5. Clusters examination

Cluster 0

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(10)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,North York,0,Italian Restaurant,Coffee Shop,Restaurant,Pet Store,Fast Food Restaurant,Sandwich Place,Pizza Place,Bank,Sushi Restaurant,Pub
5,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Park,Beer Bar,Seafood Restaurant,Pub,Italian Restaurant
7,West Toronto,0,Café,Restaurant,Bar,Coffee Shop,Bakery,Tibetan Restaurant,Gift Shop,Park,Furniture / Home Store,Vegetarian / Vegan Restaurant
8,East Toronto,0,Clothing Store,Restaurant,Coffee Shop,Bakery,Bank,Gym / Fitness Center,Sporting Goods Shop,Gas Station,Pharmacy,Tea Room
9,Downtown Toronto,0,Coffee Shop,Café,Gym,Yoga Studio,Restaurant,Italian Restaurant,Park,Dessert Shop,Bakery,French Restaurant
10,York,0,Coffee Shop,Beer Store,Discount Store,Café,BBQ Joint,Mexican Restaurant,Bus Stop,Japanese Restaurant,Bakery,Park
13,Downtown Toronto,0,Coffee Shop,Café,Clothing Store,Sushi Restaurant,Italian Restaurant,Middle Eastern Restaurant,Park,Hotel,Steakhouse,Department Store
14,Downtown Toronto,0,Korean Restaurant,Café,Coffee Shop,Grocery Store,Ice Cream Shop,Cocktail Bar,Mexican Restaurant,Pizza Place,Ethiopian Restaurant,Restaurant
15,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Park,Ramen Restaurant,Restaurant,Café,Sushi Restaurant,Gay Bar,Bubble Tea Shop,Ice Cream Shop
18,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Concert Hall,Clothing Store,Japanese Restaurant,Theater,Gym,Plaza


Cluster 1

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(10)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,Scarborough,1,Zoo Exhibit,Trail,Fast Food Restaurant,Other Great Outdoors,Elementary School,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


Cluster 2

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(10)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2,Chinese Restaurant,Shopping Mall,Bakery,Sandwich Place,Coffee Shop,Sushi Restaurant,Badminton Court,Bank,Restaurant,Cantonese Restaurant
1,Etobicoke,2,Park,Pizza Place,Discount Store,Pool,Garden Center,Gas Station,Sandwich Place,Donut Shop,Liquor Store,Italian Restaurant
2,North York,2,Pizza Place,Coffee Shop,Bank,Gas Station,Park,Shopping Mall,French Restaurant,Supermarket,Sandwich Place,Diner
12,Scarborough,2,Indian Restaurant,Coffee Shop,Bakery,Chinese Restaurant,Bank,Fried Chicken Joint,Gas Station,Hakka Restaurant,Athletics & Sports,Burger Joint
16,Scarborough,2,Coffee Shop,Pizza Place,Sandwich Place,Convenience Store,Bank,Fast Food Restaurant,Gas Station,Bakery,Fried Chicken Joint,Mexican Restaurant
17,Scarborough,2,Ice Cream Shop,Park,Coffee Shop,Discount Store,Gas Station,Gift Shop,Restaurant,Liquor Store,Trail,Bank
21,York,2,Coffee Shop,Pizza Place,Furniture / Home Store,Discount Store,Playground,Fast Food Restaurant,Gas Station,Bus Line,Grocery Store,Electronics Store
25,North York,2,Athletics & Sports,Coffee Shop,Vietnamese Restaurant,Pizza Place,Turkish Restaurant,Discount Store,Racetrack,Bank,Fast Food Restaurant,Hotel
26,North York,2,Athletics & Sports,Coffee Shop,Vietnamese Restaurant,Pizza Place,Turkish Restaurant,Discount Store,Racetrack,Bank,Fast Food Restaurant,Hotel
27,North York,2,Athletics & Sports,Coffee Shop,Vietnamese Restaurant,Pizza Place,Turkish Restaurant,Discount Store,Racetrack,Bank,Fast Food Restaurant,Hotel


Cluster 3

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(10)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
102,North York,3,Park,Baseball Field,Soccer Field,Zoo Exhibit,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School


Cluster 4

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(10)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,4,Park,Café,Golf Course,Bank,Trail,Japanese Restaurant,Chinese Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant
6,Scarborough,4,Park,Ice Cream Shop,Skating Rink,Café,Gym Pool,Restaurant,Gym,General Entertainment,Convenience Store,Thai Restaurant
42,North York,4,Furniture / Home Store,General Entertainment,Skating Rink,Electronics Store,Café,Zoo Exhibit,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
70,Downtown Toronto,4,Park,Trail,Other Great Outdoors,Scenic Lookout,Candy Store,Grocery Store,Café,Historic Site,Farmers Market,Playground
72,Scarborough,4,Home Service,Park,Gym / Fitness Center,Zoo Exhibit,Elementary School,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
92,Etobicoke,4,Park,Convenience Store,Hotel,Pizza Place,Restaurant,Gym,Café,Clothing Store,Fish & Chips Shop,Mexican Restaurant
